## Homework 6 - Numerical 

In [3]:
import math
import scipy.stats as ss 
import numpy as np 
import matplotlib.pyplot as plt 
plt.style.use('seaborn')
%matplotlib inline

In [4]:
np.random.seed(0)

In [5]:
# time index for the data 
time_index = np.arange(0.01, 1.01, 0.001)

In [6]:
# time incrementing per each value of t
time_increment = time_index[1:] - time_index[:-1]

Simulate 100 paths for the diffusions below on $[0,1]$ using the Euler scheme (see Chapter 7) for a discretization of $0.01$

In [7]:
# standard brownian motion covariance matrix 
bm_cov = np.reshape(np.array([i if i < j else j for j in time_index for i in time_index]), (1000,1000))

In [8]:
# the standard brownian motion to be implemented 
brownian_motion = np.insert(np.array([np.linalg.cholesky(bm_cov).dot(np.random.normal(loc=0, 
                                                                                      scale=1, 
                                                                                      size=1000)) for _ in range(1000)]), 
                                      obj=0, values=0, axis=1)

In [9]:
brownian_increments = np.delete(np.roll(brownian_motion,-1) - brownian_motion, obj=1000, axis=1)

In [10]:
brownian_increments

array([[ 0.17640523,  0.01265408,  0.03095041, ...,  0.00297862,
        -0.03629064, -0.01132456],
       [ 0.05559627,  0.0282225 , -0.01335477, ...,  0.00501012,
        -0.03611009, -0.04145652],
       [-0.15329211, -0.05413725,  0.00145892, ..., -0.00608436,
        -0.03834311, -0.00254875],
       ...,
       [ 0.04362489, -0.04102202,  0.06315627, ..., -0.01330407,
        -0.00344507, -0.0142965 ],
       [-0.01003402, -0.00424926, -0.00476249, ...,  0.01110308,
         0.04304031, -0.04461615],
       [-0.12446885, -0.0380794 ,  0.03138746, ..., -0.02521229,
         0.02766222,  0.04338109]])

Consider the Black-Scholes model under its risk-neutral probability

$$ d\tilde{S_t} = \sigma \tilde{S_t} d\tilde{B_t}, \ \ \ \ \ \ \  S_0 = 100 \ \ \ \ \ \ \   \sigma=0.3$$

for some Brownian motion $(\tilde{B_t})$. For simplicity, assume that the interest rate is 0. Use the pricing formula $\tilde{O_0}=\mathop{{}\mathbb{E}}[\tilde{O_1}]$ to price the following options with maturity 1
using the average over 1000 paths with a discretization of 0.001

$$where \ \ \tilde{S_t}= S_0exp(\sigma \tilde{B_t} - \sigma^2 / \ 2t)$$

In [11]:
stock_tilde = 100*np.exp((0.3*np.delete(brownian_motion, 999,1)-(0.3**2/(2*time_index))))

In [12]:
def present_value(strike:float, maturity:float, risk_free:float=0.0):
    return strike*math.exp(-risk_free*maturity)

In [13]:
def black_scholes_call(stock:float, strike:float, sigma:float, maturity:float, risk_free:float=0.0):
    vol_time = sigma*math.sqrt(maturity)
    d1 = (1/vol_time) * (math.log(stock/strike) + (risk_free+(sigma**2/2))*maturity)
    d2 = d1 - vol_time
    
    return ss.norm.cdf(d1)*stock - ss.norm.cdf(d2)*present_value(strike, maturity, risk_free)

In [24]:
# for calculating the itterative expression
def sFunc(brownian:np.array, sigma:int, initial:int, size:int):
    temp = initial
    new_arr = [temp]*(size+1)
    for i in range(size):
        new_arr[i+1] = temp + (temp * brownian[i] * sigma)  
        temp += (temp * brownian[i] * sigma)  
    return np.array(new_arr) 

In [30]:
stock_plot = np.apply_along_axis(func1d=sFunc, axis=1, 
                                 arr=np.delete(brownian_increments, 999, 1), 
                                 sigma=0.3, initial=100, size=999)

### Standard Call Option
$O_1 = (S_1 - 110)^+$

In [31]:
# simple call option
p1 = round(black_scholes_call(100, 110, 0.3, 1, 0), 4)

In [32]:
# take terminal price of the stock path 
s1 = np.reshape(stock_plot, (1000,1000))[:,-1]

In [33]:
# simulated price for call option, taking average of payouts
pA = round(np.average(np.array([i if i>0 else 0 for i in (s1-110)])), 4)

In [34]:
print("-------------------------------------------------------------------")
print("The price of the option according to black-scholes is:   ${}\nOur simulation yeilded a price of                        ${}".format(p1, pA))
print("-------------------------------------------------------------------")

-------------------------------------------------------------------
The price of the option according to black-scholes is:   $8.141
Our simulation yeilded a price of                        $8.8015
-------------------------------------------------------------------


### Lookback Option
$O_1 = max_{t\leq 1}S_t$

In [35]:
pB = round(np.average(np.array([max(i) for i in stock_plot])), 5)

In [36]:
print("-------------------------------------------------------------------")
print("The price of the option according to the simulation is {}.".format(pB))
print("-------------------------------------------------------------------")

-------------------------------------------------------------------
The price of the option according to the simulation is 126.03492.
-------------------------------------------------------------------


### Asian Option 
$O_1 = exp(\int^{1}_{0}log S_t \ dt)$

In [50]:
pC = np.average(np.exp((1/stock_plot[:,-1])-(1/stock_plot[:,0])))

In [51]:
print("-------------------------------------------------------------------")
print("The price of the option according to the simulation is {}.".format(pC))
print("-------------------------------------------------------------------")

-------------------------------------------------------------------
The price of the option according to the simulation is 1.0007792390647687.
-------------------------------------------------------------------
